# Test Dynamic Route Prompts

## Define Functions

In [1]:
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

# Function with one argument
def get_time(timezone: str) -> str:
    """Finds the current time in a specific timezone.

    :param timezone: The timezone to find the current time in, should
        be a valid timezone from the IANA Time Zone Database like
        "America/New_York" or "Europe/London". Do NOT put the place
        name itself like "rome", or "new york", you must provide
        the IANA format.
    :type timezone: str
    :return: The current time in the specified timezone."""
    now = datetime.now(ZoneInfo(timezone))
    return now.strftime("%H:%M")



def get_time_difference(timezone1: str, timezone2: str) -> str:
    """Calculates the time difference between two timezones.
    :param timezone1: The first timezone, should be a valid timezone from the IANA Time Zone Database like "America/New_York" or "Europe/London".
    :param timezone2: The second timezone, should be a valid timezone from the IANA Time Zone Database like "America/New_York" or "Europe/London".
    :type timezone1: str
    :type timezone2: str
    :return: The time difference in hours between the two timezones."""
    # Get the current time in UTC
    now_utc = datetime.utcnow().replace(tzinfo=ZoneInfo('UTC'))
    
    # Convert the UTC time to the specified timezones
    tz1_time = now_utc.astimezone(ZoneInfo(timezone1))
    tz2_time = now_utc.astimezone(ZoneInfo(timezone2))
    
    # Calculate the difference in offsets from UTC
    tz1_offset = tz1_time.utcoffset().total_seconds()
    tz2_offset = tz2_time.utcoffset().total_seconds()
    
    # Calculate the difference in hours
    hours_difference = (tz2_offset - tz1_offset) / 3600
    
    return f"The time difference between {timezone1} and {timezone2} is {hours_difference} hours."

# Function with three arguments
def convert_time(time: str, from_timezone: str, to_timezone: str) -> str:
    """Converts a specific time from one timezone to another.
    :param time: The time to convert in HH:MM format.
    :param from_timezone: The original timezone of the time, should be a valid IANA timezone.
    :param to_timezone: The target timezone for the time, should be a valid IANA timezone.
    :type time: str
    :type from_timezone: str
    :type to_timezone: str
    :return: The converted time in the target timezone.
    :raises ValueError: If the time format or timezone strings are invalid.
    
    Example:
        convert_time("12:30", "America/New_York", "Asia/Tokyo") -> "03:30"
    """
    try:
        # Use today's date to avoid historical timezone issues
        today = datetime.now().date()
        datetime_string = f"{today} {time}"
        time_obj = datetime.strptime(datetime_string, "%Y-%m-%d %H:%M").replace(tzinfo=ZoneInfo(from_timezone))
        
        converted_time = time_obj.astimezone(ZoneInfo(to_timezone))
        
        formatted_time = converted_time.strftime("%H:%M")
        return formatted_time
    except Exception as e:
        raise ValueError(f"Error converting time: {e}")

In [2]:
functions = [get_time, get_time_difference, convert_time]

## Get Function Schemas

In [3]:
from semantic_router.utils.function_call import get_schema_list

schemas = get_schema_list(functions)
schemas

c:\Users\Siraj\Documents\Personal\Work\Aurelio\Virtual Environments\semantic_router_3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[{'name': 'get_time',
  'description': 'Finds the current time in a specific timezone.\n\n:param timezone: The timezone to find the current time in, should\n    be a valid timezone from the IANA Time Zone Database like\n    "America/New_York" or "Europe/London". Do NOT put the place\n    name itself like "rome", or "new york", you must provide\n    the IANA format.\n:type timezone: str\n:return: The current time in the specified timezone.',
  'signature': '(timezone: str) -> str',
  'output': "<class 'str'>"},
 {'name': 'get_time_difference',
  'description': 'Calculates the time difference between two timezones.\n:param timezone1: The first timezone, should be a valid timezone from the IANA Time Zone Database like "America/New_York" or "Europe/London".\n:param timezone2: The second timezone, should be a valid timezone from the IANA Time Zone Database like "America/New_York" or "Europe/London".\n:type timezone1: str\n:type timezone2: str\n:return: The time difference in hours between t

In [4]:
# Generate schemas for all functions
# from semantic_router.llms.openai import get_schemas_openai
# schemas = get_schemas_openai(functions)
# schemas


## Define Routes

In [5]:
from semantic_router import Route


# Define the dynamic route with multiple functions
timezone_management_route = Route(
    name="timezone_management",
    utterances=[
        # Utterances for get_time function
        "what is the time in New York?",
        "current time in Berlin?",
        "tell me the time in Moscow right now",
        "can you show me the current time in Tokyo?",
        "please provide the current time in London",

        # Utterances for get_time_difference function
        "how many hours ahead is Tokyo from London?",
        "time difference between Sydney and Cairo",
        "what's the time gap between Los Angeles and New York?",
        "how much time difference is there between Paris and Sydney?",
        "calculate the time difference between Dubai and Toronto",

        # Utterances for convert_time function
        "convert 15:00 from New York time to Berlin time",
        "change 09:00 from Paris time to Moscow time",
        "adjust 20:00 from Rome time to London time",
        "convert 12:00 from Madrid time to Chicago time",
        "change 18:00 from Beijing time to Los Angeles time",

        # get_time and get_time_difference
        "What is the current time in Miami and how many hours is it behind London?",
        "Can you tell me the time in Tokyo and the time difference with New York?",
        "Please provide the current time in Paris and the time gap between Paris and Sydney.",
        "Show me the current time in Los Angeles and calculate how much time difference there is with Tokyo.",
        "What's the time now in Dubai and how many hours ahead is it from Toronto?",

        # get_time and convert_time
        "What is the current time in Rome? And could you convert 20:58 Rome time to Tokyo time?",
        "Can you show me the current time in Madrid? Then, convert 13:00 from Madrid time to New York time.",
        "Tell me the current time in Beijing? Following that, change 22:00 from Beijing time to London time.",
        "Please provide the current time in Sydney? Next, convert 07:30 from Sydney time to Los Angeles time.",
        "Display the current time in Toronto? Additionally, adjust 15:45 from Toronto time to Paris time.",

        # get_time_difference and convert_time
        "Calculate the time difference between New York and London and convert 09:00 New York time to London time.",
        "Find out how many hours ahead Tokyo is from San Francisco and change 15:00 from San Francisco time to Tokyo time.",
        "Determine the time gap between Sydney and Berlin and adjust 21:00 Sydney time to Berlin time.",
        "What is the time difference between Cairo and Paris, and convert 18:00 Cairo time to Paris time?",
        "Assess the hours difference between Chicago and Madrid and convert 12:00 Chicago time to Madrid time.",

        # All three functions
        "What is the time in Seattle? What is the time difference between Mumbai and Tokyo? What is 5:53 Toronto time in Sydney time?"
        "What's the current time in San Francisco? How many hours behind is San Francisco from Berlin? Convert 14:00 from San Francisco time to Berlin time.",
        "Tell me the time in Cairo, the time difference between Cairo and Tokyo, and convert 16:00 from Cairo time to Tokyo time.",
        "Can you provide the current time in Sydney, the time gap between Sydney and London, and change 19:00 from Sydney time to London time?",
        "Show me the current time in Chicago, calculate the time difference between Chicago and Paris, and adjust 10:00 from Chicago time to Paris time."
    ],
    function_schemas=schemas
)

routes = [timezone_management_route]

## Define Encoder

In [6]:
import os
from getpass import getpass
from semantic_router import RouteLayer
from semantic_router.encoders import CohereEncoder, OpenAIEncoder

# dashboard.cohere.ai
# os.environ["COHERE_API_KEY"] = os.getenv("COHERE_API_KEY") or getpass(
#     "Enter Cohere API Key: "
# )
# platform.openai.com
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass(
    "Enter OpenAI API Key: "
)

# encoder = CohereEncoder()
encoder = OpenAIEncoder()

## Define Route Layer

In [7]:
rl = RouteLayer(encoder=encoder, routes=routes)

2024-05-03 02:35:19 INFO semantic_router.utils.logger local
2024-05-03 02:35:19 INFO semantic_router.utils.logger Document 1 length: 29
2024-05-03 02:35:19 INFO semantic_router.utils.logger Document 1 trunc length: 29
2024-05-03 02:35:19 INFO semantic_router.utils.logger Document 2 length: 23
2024-05-03 02:35:19 INFO semantic_router.utils.logger Document 2 trunc length: 23
2024-05-03 02:35:19 INFO semantic_router.utils.logger Document 3 length: 36
2024-05-03 02:35:19 INFO semantic_router.utils.logger Document 3 trunc length: 36
2024-05-03 02:35:19 INFO semantic_router.utils.logger Document 4 length: 42
2024-05-03 02:35:19 INFO semantic_router.utils.logger Document 4 trunc length: 42
2024-05-03 02:35:19 INFO semantic_router.utils.logger Document 5 length: 41
2024-05-03 02:35:19 INFO semantic_router.utils.logger Document 5 trunc length: 41
2024-05-03 02:35:19 INFO semantic_router.utils.logger Document 6 length: 42
2024-05-03 02:35:19 INFO semantic_router.utils.logger Document 6 trunc len

## Define Test Utterances

In [8]:
test_utterances_get_time = [
    "Could you tell me the current time in Miami?",
    "What's the time now in Cairo?",
    "I need the time in Shanghai right now.",
    "Please show the current time in Johannesburg.",
    "What time is it currently in Buenos Aires?",
    "Can you find the time in Seoul at this moment?",
    "I'd like to know the time in Bangkok.",
    "What is the time right now in Istanbul?",
    "Could you check the time in Toronto?",
    "Please provide the time in Mexico City.",
    "Show me what time it is in Mumbai.",
    "I need to know the current time in Manila.",
    "What's the current time in Lagos?",
    "Can you tell me what time it is in Jakarta?",
    "Find out the current time in Sao Paulo.",
    "What is the time in Paris at the moment?",
    "Please tell me the time in Dubai.",
    "I'm looking for the current time in Chicago.",
    "Could you provide the time in Hong Kong?",
    "What's the time in Barcelona right now?",
    "Can you display the current time in Montreal?",
    "I want to know the time in Helsinki.",
    "Please find the current time in Zurich.",
    "Tell me the current time in Copenhagen.",
    "Show the current time in Athens.",
    "What is the current time in Las Vegas?",
    "Can you provide the current time in London?",
    "Please tell me what time it is now in Tokyo.",
    "I need the current time in Rome, please.",
    "Could you show me the time in Sydney?",
    "What time is it right now in Oslo?",
    "Can you check the current time in New Delhi?",
    "I'd like to know the current time in Rio de Janeiro.",
    "Please display the current time in Toronto.",
    "What's the time in Berlin at this moment?",
    "Show me the time in Amsterdam.",
    "Could you find out the time in Brussels?",
    "I want to know the current time in Stockholm.",
    "Please provide the time in Vienna.",
    "Tell me the current time in Prague.",
    "What is the time in Budapest right now?",
    "Can you tell me the time in Lisbon?",
    "I'm looking for the current time in Madrid.",
    "Could you check the time in Warsaw?",
    "Please show the current time in Helsinki.",
    "What's the current time in Zurich?",
    "Can you find the time in Moscow right now?",
    "I need to know the time in Beijing.",
    "Show me the current time in Seoul.",
    "What time is it in Singapore?"
]

In [9]:
test_utterances_get_time_difference = [
    "What's the hour difference between Vancouver and Miami?",
    "Can you calculate the hours separating Helsinki from Denver?",
    "Determine the time offset between Johannesburg and Rome.",
    "How many hours separate Beijing from Vancouver?",
    "Find the time difference from Madrid to Singapore.",
    "What is the hour gap between Boston and San Diego?",
    "Measure the time difference from Dubai to San Francisco.",
    "How far ahead is Melbourne compared to Ottawa?",
    "Calculate the hours between Seoul and Buenos Aires.",
    "What's the time difference from Istanbul to Toronto?",
    "Can you tell me how many hours are between Cairo and Montreal?",
    "Determine the time gap from Manila to London.",
    "How many hours ahead is Sydney from Mexico City?",
    "What is the time difference between Oslo and Los Angeles?",
    "Calculate the time offset from Paris to Tokyo.",
    "How many hours separate New Delhi from Chicago?",
    "Find out the time difference between Zurich and Seattle.",
    "What's the hour difference from Lisbon to Hong Kong?",
    "Measure the time gap between Stockholm and Houston.",
    "Determine the hours separating Amsterdam from Shanghai.",
    "Calculate the time difference from Brussels to Las Vegas.",
    "How many hours ahead is Copenhagen from Phoenix?",
    "What is the time difference between Warsaw and Atlanta?",
    "Can you find the time gap from Budapest to Miami?",
    "Tell me the hours separating Kuala Lumpur and New York.",
    "What is the time difference between Toronto and Vancouver?",
    "Can you show the time gap between Mexico City and Toronto?",
    "How many hours are between London and New York?",
    "Please calculate the time difference from Tokyo to San Francisco.",
    "Determine the hours separating Paris from Montreal.",
    "What's the hour difference between Dubai and London?",
    "Measure the time gap from Seoul to Los Angeles.",
    "How far ahead is Moscow compared to Lisbon?",
    "Calculate the hours between Rome and Chicago.",
    "What's the time difference from Bangkok to Johannesburg?",
    "Can you tell me how many hours are between Mumbai and Dubai?",
    "Determine the time gap from Beijing to Sydney.",
    "How many hours ahead is Cairo from Buenos Aires?",
    "What is the time difference between Helsinki and Denver?",
    "Calculate the time offset from Zurich to Cairo.",
    "How many hours separate Stockholm from Tokyo?",
    "Find out the time difference between Madrid and Beijing.",
    "What's the hour difference from Amsterdam to Miami?",
    "Measure the time gap between Houston and Berlin.",
    "Determine the hours separating Singapore from London.",
    "Calculate the time difference from Athens to New Delhi.",
    "How many hours ahead is Brussels from San Diego?",
    "What is the time difference between Copenhagen and Toronto?",
    "Can you find the time gap from Oslo to Rome?",
    "Tell me the hours separating Montreal and Las Vegas."
]

In [10]:
test_utterances_convert_time = [
    "Please switch 14:30 from Los Angeles time to Tokyo time.",
    "Modify 10:15 from London time to New York time.",
    "Shift 23:00 from Hong Kong time to Paris time.",
    "Transform 08:45 from Sydney time to Moscow time.",
    "Reformat 17:20 from Cairo time to Berlin time.",
    "Alter 12:00 from Chicago time to Dubai time.",
    "Revise 19:30 from Toronto time to Seoul time.",
    "Update 16:00 from Madrid time to Singapore time.",
    "Change 11:00 from Rome time to Johannesburg time.",
    "Convert 22:15 from Mumbai time to Vancouver time.",
    "Adjust 07:50 from Bangkok time to San Francisco time.",
    "Switch 13:30 from Montreal time to Lisbon time.",
    "Modify 15:45 from Amsterdam time to Los Angeles time.",
    "Shift 09:00 from Zurich time to Beijing time.",
    "Transform 20:30 from Buenos Aires time to London time.",
    "Reformat 18:00 from Seattle time to Athens time.",
    "Alter 21:45 from Mexico City time to Helsinki time.",
    "Revise 05:30 from Copenhagen time to Rio de Janeiro time.",
    "Update 14:20 from Stockholm time to Cairo time.",
    "Change 16:50 from Dublin time to Tokyo time.",
    "Convert 12:30 from Brussels time to New Delhi time.",
    "Adjust 10:00 from Vienna time to Shanghai time.",
    "Switch 22:00 from Oslo time to Istanbul time.",
    "Modify 08:00 from Helsinki time to Mexico City time.",
    "Shift 17:15 from Prague time to Sydney time.",
    "Reconfigure 13:00 from Paris time to Los Angeles time.",
    "Rework 18:45 from Tokyo time to Rome time.",
    "Adapt 06:30 from New York time to Dubai time.",
    "Rearrange 21:15 from Berlin time to Mumbai time.",
    "Recalculate 15:30 from San Francisco time to London time.",
    "Realign 22:00 from Seoul time to New York time.",
    "Reassign 11:45 from Lisbon time to Beijing time.",
    "Reform 20:25 from Johannesburg time to Chicago time.",
    "Reschedule 09:50 from Athens time to Toronto time.",
    "Replot 14:10 from Montreal time to Cairo time.",
    "Rephrase 16:40 from Houston time to Paris time.",
    "Reposition 12:20 from Madrid time to Sydney time.",
    "Reevaluate 07:55 from Singapore time to Vancouver time.",
    "Redirect 19:00 from Rome time to Montreal time.",
    "Revisit 10:30 from Dubai time to San Francisco time.",
    "Refine 23:45 from Beijing time to Zurich time.",
    "Remodel 08:10 from London time to Seoul time.",
    "Revamp 17:00 from Sydney time to Madrid time.",
    "Rehash 15:20 from Copenhagen time to Tokyo time.",
    "Reinvent 22:30 from Las Vegas time to Amsterdam time.",
    "Rebuild 13:15 from Rio de Janeiro time to Stockholm time.",
    "Recreate 18:55 from Zurich time to Hong Kong time.",
    "Redesign 21:40 from Chicago time to Athens time.",
    "Reestablish 12:00 from Toronto time to Lisbon time.",
    "Reformulate 20:00 from Moscow time to Buenos Aires time."
]

In [11]:
test_utterances_get_time_and_get_time_difference = [
    "What's the current time in Vancouver and how far behind is it from Tokyo?",
    "Please tell me the current time in Helsinki and the hour difference with Cairo.",
    "Can you find out the time in Sao Paulo and how many hours it is ahead of London?",
    "I need the current time in Manila and the time difference to Berlin.",
    "Show the current time in Zurich and calculate the time gap with Sydney.",
    "Could you provide the time in Istanbul and tell me how many hours it is behind New York?",
    "What is the time in Oslo right now and how many hours is it ahead of Montreal?",
    "Can you display the current time in Mexico City and the hours difference with Tokyo?",
    "Tell me the current time in Lisbon and how many hours it is behind Beijing.",
    "I'd like to know the current time in Budapest and the time difference with Los Angeles.",
    "Please show the current time in Stockholm and the hour gap to Dubai.",
    "What's the time in Houston right now and how many hours ahead is it from Paris?",
    "Can you check the current time in Atlanta and how far behind it is from Moscow?",
    "Find out the current time in Denver and the time difference to Singapore.",
    "I want to know the current time in Mumbai and how many hours it is ahead of Rio de Janeiro.",
    "Please provide the current time in Seattle and the time gap with London.",
    "Show me the current time in Johannesburg and how many hours it is behind Tokyo.",
    "Could you tell me the current time in Phoenix and the hour difference with Cairo?",
    "What is the current time in Brussels and how many hours is it ahead of Vancouver?",
    "Can you find the current time in Dublin and how many hours it is behind Hong Kong?",
    "Tell me the current time in Las Vegas and the time difference with Berlin.",
    "I need the current time in Toronto and how many hours it is ahead of Madrid.",
    "Please display the current time in Montreal and the hours difference with Sydney.",
    "What's the current time in Copenhagen and how many hours behind is it from Tokyo?",
    "Can you provide the current time in Athens and tell me the time difference with New York?",
    "Could you check the current time in Prague and how many hours it is ahead of Denver?",
    "What is the current time in Warsaw and the time difference with San Francisco?",
    "Please find out the current time in Bangkok and how far behind it is from London.",
    "Show me the current time in Kuala Lumpur and calculate the hours difference to New York.",
    "Can you tell me the current time in Rome and how many hours it is behind Tokyo?",
    "I need the current time in Beijing and the time gap with Los Angeles.",
    "What's the current time in Santiago and how many hours ahead is it from Cairo?",
    "Please provide the current time in Lagos and the hour difference with Paris.",
    "Tell me the current time in Melbourne and how many hours it is behind Moscow.",
    "Can you display the current time in Caracas and the hours difference with Berlin?",
    "Find out the current time in Tehran and the time difference to Sydney.",
    "I'd like to know the current time in Havana and how many hours it is ahead of Toronto.",
    "What is the current time in Bogota and how many hours behind is it from Seoul?",
    "Could you provide the current time in Nairobi and tell me the time gap with Dubai?",
    "Show the current time in Lima and calculate the hours difference to Madrid.",
    "Please check the current time in Dhaka and how far behind it is from New York.",
    "Can you find the current time in Hanoi and the hour difference with London?",
    "Tell me the current time in Riyadh and how many hours it is ahead of Montreal.",
    "What's the current time in Belgrade and the time difference with San Francisco?",
    "I want to know the current time in Quito and how many hours it is behind Tokyo.",
    "Could you tell me the current time in Antwerp and the hour gap with Los Angeles?",
    "Please show the current time in Edinburgh and how many hours ahead it is from Cairo.",
    "Find out the current time in Casablanca and the time difference to Beijing.",
    "Can you provide the current time in Accra and how many hours it is behind Paris?",
    "What is the current time in Algiers and how many hours ahead is it from Vancouver?"
]

In [12]:
test_utterances_get_time_and_convert_time = [
    "Could you provide the current time in Vancouver? Additionally, what is 15:00 Vancouver time in Tokyo time?",
    "What's the current time in Oslo? And can you convert 14:00 from Oslo time to New York time?",
    "I need the current time in Mexico City. Please convert 09:30 Mexico City time to London time.",
    "Show me the current time in Helsinki. Also, change 16:30 from Helsinki time to Beijing time.",
    "Can you tell me the current time in Cairo? Then, adjust 11:15 from Cairo time to Paris time.",
    "Please find the current time in Lisbon. Convert 10:00 from Lisbon time to Sydney time.",
    "What is the current time in Budapest? Can you switch 13:20 from Budapest time to Los Angeles time?",
    "Display the current time in Jakarta. Change 18:00 Jakarta time to Moscow time.",
    "Tell me the current time in Seoul. Convert 09:45 from Seoul time to Berlin time.",
    "Can you show the current time in Zurich? And adjust 12:00 from Zurich time to Dubai time?",
    "Please provide the current time in Toronto. Convert 20:00 from Toronto time to Madrid time.",
    "What's the current time in Mumbai? Can you change 15:30 from Mumbai time to Chicago time?",
    "Find the current time in Rio de Janeiro. Switch it to San Francisco time for 22:00.",
    "Could you check the current time in Montreal? Convert 07:00 from Montreal time to Istanbul time.",
    "I'd like to know the current time in Manila. Adjust 17:50 from Manila time to Athens time.",
    "Please tell me the current time in Dubai. Convert 19:25 from Dubai time to Vancouver time.",
    "Show the current time in Barcelona. Change 08:15 from Barcelona time to Tokyo time.",
    "Can you provide the current time in Houston? Switch 12:30 from Houston time to Paris time.",
    "What is the current time in Prague? Can you convert 14:50 from Prague time to Seoul time?",
    "Display the current time in Stockholm. Adjust 11:00 from Stockholm time to Johannesburg time.",
    "Tell me the current time in Las Vegas. Convert 16:40 from Las Vegas time to Rome time.",
    "Can you find the current time in San Diego? Change 13:00 from San Diego time to Hong Kong time.",
    "Please show the current time in Brussels. Convert 18:30 from Brussels time to Sydney time.",
    "What's the current time in Copenhagen? Can you adjust 15:20 from Copenhagen time to New Delhi time?",
    "Find out the current time in Warsaw. Switch 20:00 from Warsaw time to Los Angeles time.",
    "Could you show the current time in Frankfurt? Convert 10:00 from Frankfurt time to Beijing time.",
    "What is the current time in Atlanta? Can you switch 15:00 from Atlanta time to Paris time?",
    "Please find the current time in Cape Town. Adjust 12:45 from Cape Town time to New York time.",
    "Tell me the current time in Ankara. Change 17:30 from Ankara time to London time.",
    "Can you provide the current time in Milan? Convert 14:20 from Milan time to Tokyo time.",
    "Display the current time in Phoenix. Switch 11:00 from Phoenix time to Moscow time.",
    "I need the current time in Buenos Aires. Please convert 19:00 Buenos Aires time to Seoul time.",
    "What's the current time in Dublin? Can you adjust 13:15 from Dublin time to Shanghai time?",
    "Show me the current time in Tehran. Change 16:00 from Tehran time to Berlin time.",
    "Can you tell me the current time in Bogota? Convert 18:30 from Bogota time to Sydney time.",
    "Please check the current time in Santiago. Switch 20:45 from Santiago time to Los Angeles time.",
    "Find out the current time in Lagos. Adjust 09:00 from Lagos time to Hong Kong time.",
    "Could you provide the current time in Perth? Convert 22:15 from Perth time to Madrid time.",
    "What is the current time in Beirut? Can you change 07:30 from Beirut time to Toronto time?",
    "Display the current time in Kiev. Switch 12:50 from Kiev time to Vancouver time.",
    "Tell me the current time in Edinburgh. Convert 15:40 from Edinburgh time to Dubai time.",
    "Can you show the current time in Belgrade? Adjust 14:25 from Belgrade time to Singapore time.",
    "Please provide the current time in Vienna. Change 17:00 from Vienna time to Johannesburg time.",
    "What's the current time in Oslo? Can you convert 19:00 from Oslo time to Cairo time?",
    "Find the current time in Havana. Switch it to Melbourne time for 21:30.",
    "Could you check the current time in Dhaka? Convert 08:45 from Dhaka time to Rome time.",
    "I'd like to know the current time in Quito. Adjust 10:30 from Quito time to Paris time.",
    "Please tell me the current time in Antwerp. Change 13:50 from Antwerp time to Beijing time.",
    "Show the current time in Glasgow. Convert 11:15 from Glasgow time to New Delhi time.",
    "Can you find the current time in Caracas? Switch 16:00 from Caracas time to Tokyo time."
]

In [13]:
test_utterances_get_time_difference_and_convert_time = [
    "What's the time difference between Los Angeles and Tokyo, and can you convert 10:00 from Los Angeles to Tokyo time?",
    "Please find out how many hours ahead London is from New York and switch 14:30 from London to New York time.",
    "Can you determine the hour gap between Paris and Sydney, and adjust 16:00 Paris time to Sydney time?",
    "Show the time difference from Rome to Beijing and change 18:15 from Rome to Beijing time.",
    "Calculate how far ahead Mumbai is from Toronto and convert 20:00 Mumbai time to Toronto time.",
    "Find out the hours difference between Seoul and London, and convert 09:00 Seoul time to London time.",
    "What is the time gap between Chicago and Dubai, and can you switch 12:45 from Chicago to Dubai time?",
    "Determine how many hours are between Johannesburg and New York, and adjust 11:00 Johannesburg time to New York time.",
    "Measure the time difference from Bangkok to Los Angeles and convert 13:30 Bangkok time to Los Angeles time.",
    "Assess the hour difference between Vancouver and Paris, and change 15:00 Vancouver time to Paris time.",
    "Calculate the time offset between Houston and Tokyo, and switch 17:00 Houston time to Tokyo time.",
    "How many hours separate Cairo from Berlin? Please convert 19:00 Cairo time to Berlin time.",
    "What's the time difference from Singapore to San Francisco, and can you adjust 21:00 Singapore time to San Francisco time?",
    "Determine the hours separating Melbourne from Madrid, and convert 08:00 Melbourne time to Madrid time.",
    "Find the time gap between Zurich and Sydney, and change 10:15 Zurich time to Sydney time.",
    "How far ahead is Helsinki compared to Denver? Convert 12:30 Helsinki time to Denver time.",
    "What is the hour difference between Oslo and Los Angeles, and can you convert 14:45 Oslo time to Los Angeles time?",
    "Measure the time difference from Montreal to Dubai and switch 16:20 Montreal time to Dubai time.",
    "Can you tell me how many hours are between Amsterdam and Beijing, and adjust 18:00 Amsterdam time to Beijing time?",
    "Determine the time gap from Brussels to Johannesburg and convert 20:30 Brussels time to Johannesburg time.",
    "Calculate the hours between Stockholm and Houston, and change 07:00 Stockholm time to Houston time.",
    "What's the time difference from Dublin to Tokyo, and can you convert 09:30 Dublin time to Tokyo time?",
    "How many hours ahead is Copenhagen from Phoenix? Please adjust 11:45 Copenhagen time to Phoenix time.",
    "Find out the time difference between Warsaw and Atlanta, and switch 13:15 Warsaw time to Atlanta time.",
    "Measure the time gap between Lisbon and Hong Kong, and convert 15:50 Lisbon time to Hong Kong time.",
    "Identify the time difference between Miami and Singapore, and convert 08:00 Miami time to Singapore time.",
    "Estimate how many hours ahead is Berlin from Los Angeles, and switch 12:00 Berlin time to Los Angeles time.",
    "Can you calculate the time gap between Tokyo and Rome, and adjust 15:30 Tokyo time to Rome time?",
    "Reveal the hours difference from Madrid to Shanghai, and change 17:45 Madrid time to Shanghai time.",
    "Compute the time offset between Toronto and Cairo, and convert 13:00 Toronto time to Cairo time.",
    "Examine the time difference from Sydney to Chicago, and switch 19:00 Sydney time to Chicago time.",
    "What's the hour gap between Dubai and San Francisco, and can you convert 11:15 Dubai time to San Francisco time?",
    "Determine the time difference from Paris to Johannesburg, and adjust 14:00 Paris time to Johannesburg time.",
    "Calculate how many hours separate Beijing from Vancouver, and change 16:30 Beijing time to Vancouver time.",
    "Assess the time offset between Seattle and London, and convert 18:00 Seattle time to London time.",
    "Find out how far ahead is Buenos Aires from Moscow, and switch 20:00 Buenos Aires time to Moscow time.",
    "What is the time difference between New Delhi and Athens, and can you adjust 09:45 New Delhi time to Athens time?",
    "Measure the hours separating Houston from Berlin, and convert 12:10 Houston time to Berlin time.",
    "Determine the time gap from Montreal to Tokyo, and change 14:25 Montreal time to Tokyo time.",
    "Calculate the hour difference between Zurich and Los Angeles, and switch 17:15 Zurich time to Los Angeles time.",
    "How many hours ahead is Stockholm from Denver? Convert 19:30 Stockholm time to Denver time.",
    "What's the time difference from Amsterdam to Cairo, and can you convert 21:00 Amsterdam time to Cairo time?",
    "Measure the time gap between Brussels and Sydney, and adjust 10:20 Brussels time to Sydney time.",
    "Can you tell me how many hours are between Lisbon and Beijing, and change 12:40 Lisbon time to Beijing time?",
    "Determine the hours separating Oslo from New York, and convert 15:00 Oslo time to New York time.",
    "Calculate the time difference from Copenhagen to Dubai, and switch 18:15 Copenhagen time to Dubai time.",
    "Find out the hour gap between Warsaw and San Francisco, and adjust 20:45 Warsaw time to San Francisco time.",
    "How far ahead is Helsinki compared to Houston? Please convert 07:30 Helsinki time to Houston time.",
    "What is the time difference between Prague and Toronto, and can you switch 09:50 Prague time to Toronto time?",
    "Measure the time gap between Budapest and Melbourne, and convert 11:55 Budapest time to Melbourne time."
]

In [14]:
test_utterances_all_functions = [
    "What's the current time in Atlanta? How many hours is Atlanta ahead of Vancouver? And convert 14:20 from Atlanta time to Vancouver time.",
    "Please tell me the current time in Boston, the time difference with Zurich, and change 16:00 Boston time to Zurich time.",
    "Can you show me the current time in Denver? What's the hour difference between Denver and Amsterdam? Also, adjust 18:30 from Denver time to Amsterdam time.",
    "I need the current time in Houston, the time gap between Houston and Singapore, and convert 20:15 Houston time to Singapore time.",
    "Display the current time in Phoenix, calculate the time difference between Phoenix and Madrid, and switch 22:00 Phoenix time to Madrid time.",
    "Tell me the current time in Toronto, how many hours behind is Toronto from Dubai, and change 09:00 Toronto time to Dubai time.",
    "What is the current time in Montreal? What is the time difference between Montreal and Beijing? And convert 11:30 Montreal time to Beijing time.",
    "Show me the current time in Miami, the hours difference with Moscow, and adjust 13:45 Miami time to Moscow time.",
    "Can you provide the current time in Seattle, the time gap between Seattle and Rome, and convert 15:10 Seattle time to Rome time?",
    "Find out the current time in San Diego, how many hours ahead is San Diego from Cairo, and switch 17:25 San Diego time to Cairo time.",
    "What's the current time in Minneapolis? How far behind is Minneapolis from Tokyo? Convert 19:00 Minneapolis time to Tokyo time.",
    "Please display the current time in Las Vegas, the hour difference with Paris, and adjust 21:15 Las Vegas time to Paris time.",
    "Can you tell me the current time in Philadelphia, the time difference between Philadelphia and London, and change 08:00 Philadelphia time to London time?",
    "I'd like to know the current time in Dallas, how many hours it is ahead of Hong Kong, and convert 10:30 Dallas time to Hong Kong time.",
    "Show the current time in San Francisco, calculate the time gap with Berlin, and switch 12:45 San Francisco time to Berlin time.",
    "Could you provide the current time in Baltimore and tell me how many hours it is behind Lisbon? Also, adjust 14:55 Baltimore time to Lisbon time.",
    "What is the current time in Chicago and how many hours ahead is it from Sydney? Please convert 16:40 Chicago time to Sydney time.",
    "Can you display the current time in New Orleans and the hours difference with Frankfurt? Change 18:20 New Orleans time to Frankfurt time.",
    "Tell me the current time in Salt Lake City and how many hours it is behind Istanbul. Convert 20:00 Salt Lake City time to Istanbul time.",
    "Please find the current time in Milwaukee, the time difference with Melbourne, and adjust 07:30 Milwaukee time to Melbourne time.",
    "What's the current time in Nashville? How far ahead is Nashville from Buenos Aires? Switch 09:15 Nashville time to Buenos Aires time.",
    "Can you show the current time in Indianapolis, the hour gap between Indianapolis and Seoul, and convert 11:00 Indianapolis time to Seoul time?",
    "Display the current time in Columbus, calculate the time difference between Columbus and Johannesburg, and change 13:20 Columbus time to Johannesburg time.",
    "Tell me the current time in Charlotte, how many hours behind is Charlotte from Dubai, and adjust 15:40 Charlotte time to Dubai time.",
    "What is the current time in Detroit? What is the time difference between Detroit and Beijing? And convert 17:55 Detroit time to Beijing time.",
    "What's the current time in London? How many hours is London ahead of Los Angeles? And convert 15:30 from London time to Los Angeles time.",
    "Please tell me the current time in Cairo, the time difference with Tokyo, and change 17:00 Cairo time to Tokyo time.",
    "Can you show me the current time in Madrid? What's the hour difference between Madrid and Beijing? Also, adjust 19:00 from Madrid time to Beijing time.",
    "I need the current time in Rome, the time gap between Rome and Sydney, and convert 21:00 Rome time to Sydney time.",
    "Display the current time in Paris, calculate the time difference between Paris and Chicago, and switch 23:00 Paris time to Chicago time.",
    "Tell me the current time in Berlin, how many hours behind is Berlin from Dubai, and change 08:30 Berlin time to Dubai time.",
    "What is the current time in Amsterdam? What is the time difference between Amsterdam and San Francisco? And convert 10:15 Amsterdam time to San Francisco time.",
    "Show me the current time in Vienna, the hours difference with Seoul, and adjust 12:00 Vienna time to Seoul time.",
    "Can you provide the current time in Zurich, the time gap between Zurich and Mumbai, and convert 14:00 Zurich time to Mumbai time?",
    "Find out the current time in Stockholm, how many hours ahead is Stockholm from Rio de Janeiro, and switch 16:00 Stockholm time to Rio de Janeiro time.",
    "What's the current time in Prague? How far behind is Prague from Singapore? Convert 18:00 Prague time to Singapore time.",
    "Please display the current time in Lisbon, the hour difference with New York, and adjust 20:00 Lisbon time to New York time.",
    "Can you tell me the current time in Budapest, the time difference between Budapest and Toronto, and change 07:45 Budapest time to Toronto time?",
    "I'd like to know the current time in Helsinki, how many hours it is ahead of Vancouver, and convert 09:30 Helsinki time to Vancouver time.",
    "Show the current time in Oslo, calculate the time gap with Mexico City, and switch 11:15 Oslo time to Mexico City time.",
    "Could you provide the current time in Athens and tell me how many hours it is behind Tokyo? Also, adjust 13:00 Athens time to Tokyo time.",
    "What is the current time in Warsaw and how many hours ahead is it from Montreal? Please convert 15:00 Warsaw time to Montreal time.",
    "Can you display the current time in Dublin and the hours difference with Istanbul? Change 17:15 Dublin time to Istanbul time.",
    "Tell me the current time in Brussels and how many hours it is behind Hong Kong. Convert 19:30 Brussels time to Hong Kong time.",
    "Please find the current time in Copenhagen, the time difference with Johannesburg, and adjust 21:45 Copenhagen time to Johannesburg time.",
    "What's the current time in Glasgow? How far ahead is Glasgow from Buenos Aires? Switch 08:00 Glasgow time to Buenos Aires time.",
    "Can you show the current time in Cardiff, the hour gap between Cardiff and Seoul, and convert 10:30 Cardiff time to Seoul time?",
    "Display the current time in Edinburgh, calculate the time difference between Edinburgh and Delhi, and change 12:45 Edinburgh time to Delhi time.",
    "Tell me the current time in Manchester, how many hours behind is Manchester from Dubai, and adjust 14:30 Manchester time to Dubai time.",
    "What is the current time in Liverpool? What is the time difference between Liverpool and Beijing? And convert 16:15 Liverpool time to Beijing time."
]

## Define Testing Functions

In [15]:
def parse_response(response: str):
    results = []
    for call in response.function_call:
        args = call['arguments']
        if call['function_name'] == 'get_time':
            result = get_time(**args)
            results.append(result)
            print(result)
        if call['function_name'] == 'get_time_difference':
            result = get_time_difference(**args)
            results.append(result)
            print(result)
        if call['function_name'] == 'convert_time':
            result = convert_time(**args)
            results.append(result)
            print(result)
    return results
    

In [16]:
def check_num_results(function_name, results):
    if function_name == 'get_time' or function_name == 'get_time_difference' or function_name == 'convert_time':
        if len(results) != 1:
            print(f"Error: {function_name} should have exactly 1 result, but got {len(results)}.")
            return False
    elif function_name == 'get_time_and_get_time_difference' or function_name == 'get_time_and_convert_time' or function_name == 'get_time_difference_and_convert_time':
        if len(results) != 2:
            print(f"Error: {function_name} should have exactly 2 results, but got {len(results)}.")
            return False
    elif function_name == 'all_functions':
        if len(results) != 3:
            print(f"Error: {function_name} should have exactly 3 results, but got {len(results)}.")
            return False
    else:
        print(f"Error: Unknown function name '{function_name}'.")
        return False
    return True


In [17]:
import re

get_time_pattern = convert_time_pattern = r"^\d{2}:\d{2}$"
get_time_difference_pattern = r"The time difference between (.+) and (.+) is ([-+]?\d+\.\d+) hours."

def results_pass(function_name: str, results: list[str]):
    if function_name == 'get_time':
        if not re.match(get_time_pattern, results[0]):
            print(f"{function_name} result '{results[0]}' does not match the expected format 'HH:MM'.")
            return False
        return True
    elif function_name == 'get_time_difference':
        if not re.match(get_time_difference_pattern, results[0]):
            print(f"{get_time_difference} result '{results[0]}' does not match the expected format.")
            return False
        return True
    elif function_name == 'convert_time':
        if not re.match(convert_time_pattern, results[0]):
            print(f"{function_name} result '{results[0]}' does not match the expected format 'HH:MM'.")
            return False
        return True
    elif function_name == 'get_time_and_get_time_difference':
        if not re.match(get_time_pattern, results[0]):
            print(f"{function_name} first result '{results[0]}' does not match the expected format 'HH:MM'.")
            return False
        if not re.match(get_time_difference_pattern, results[1]):
            print(f"{function_name} second result '{results[1]}' does not match the expected format.")
            return False
        return True
    elif function_name == 'get_time_and_convert_time':
        if not re.match(get_time_pattern, results[0]):
            print(f"{function_name} first result '{results[0]}' does not match the expected format 'HH:MM'.")
            return False
        if not re.match(convert_time_pattern, results[1]):
            print(f"{function_name} second result '{results[1]}' does not match the expected format 'HH:MM'.")
            return False
        return True
    elif function_name == 'get_time_difference_and_convert_time':
        if not re.match(get_time_difference_pattern, results[0]):
            print(f"{function_name} first result '{results[0]}' does not match the expected format.")
            return False
        if not re.match(convert_time_pattern, results[1]):
            print(f"{function_name} second result '{results[1]}' does not match the expected format 'HH:MM'.")
            return False
        return True
    elif function_name == 'all_functions':
        if not re.match(get_time_pattern, results[0]):
            print(f"{function_name} first result '{results[0]}' does not match the expected format 'HH:MM'.")
            return False
        if not re.match(get_time_difference_pattern, results[1]):
            print(f"{function_name} second result '{results[1]}' does not match the expected format.")
            return False
        if not re.match(convert_time_pattern, results[2]):
            print(f"{function_name} third result '{results[2]}' does not match the expected format.")
            return False
        return True
    else:
        raise ValueError(f"Invalid function name '{function_name}'.")

In [18]:
def test_function_calling(utterances, function_name, expected_num_responses=1):
    success_count = 0
    fail_count = 0
    for utterance in utterances:
        print('='*100)
        print(u"Processing utterance: ", utterance)
        try:
            result = rl(utterance)
            results = parse_response(result)
            if not check_num_results(function_name, results):
                print('>>> FAIL <<<.')
                print(f"Number of results does not match expected for {function_name}")
                fail_count += 1
            elif not results_pass(function_name, results):
                print('>>> FAIL <<<.')
                print(f"Results format does not match expected for {function_name}")
                fail_count += 1
            else:
                print('>>> PASS <<<.')
                success_count += 1
        except Exception as e:
            print('>>> FAIL <<<.')
            print(f"Failed to process: {utterance} with error: {e}")
            fail_count += 1
        print('='*100)
    print('='*100)
    print(f"Number of successful responses: {success_count}")
    print(f"Number of failed responses: {fail_count}")
    print('='*100)

## Run Tests

### Test `get_time` Function Argument Extraction

In [19]:
test_function_calling(
    utterances=test_utterances_get_time,
    function_name='get_time',
    expected_num_responses=1
    )

2024-05-03 02:35:21 INFO semantic_router.utils.logger Document 1 length: 44
2024-05-03 02:35:21 INFO semantic_router.utils.logger Document 1 trunc length: 44


Processing utterance:  Could you tell me the current time in Miami?


2024-05-03 02:35:21 WARNING semantic_router.utils.logger No LLM provided for dynamic route, will use OpenAI LLM default. Ensure API key is set in OPENAI_API_KEY environment variable.
2024-05-03 02:35:21 INFO semantic_router.utils.logger Extracting function input...
2024-05-03 02:35:22 INFO semantic_router.utils.logger LLM output: [
    {
        "function_name": "get_time",
        "arguments": {
            "timezone": "America/New_York"
        }
    }
]
2024-05-03 02:35:22 INFO semantic_router.utils.logger Function inputs: [{'function_name': 'get_time', 'arguments': {'timezone': 'America/New_York'}}]
2024-05-03 02:35:22 INFO semantic_router.utils.logger Document 1 length: 29
2024-05-03 02:35:22 INFO semantic_router.utils.logger Document 1 trunc length: 29


18:35
>>> PASS <<<.
Processing utterance:  What's the time now in Cairo?


2024-05-03 02:35:23 INFO semantic_router.utils.logger Extracting function input...
2024-05-03 02:35:24 INFO semantic_router.utils.logger LLM output: [
    {
        "function_name": "get_time",
        "arguments": {
            "timezone": "Africa/Cairo"
        }
    }
]
2024-05-03 02:35:24 INFO semantic_router.utils.logger Function inputs: [{'function_name': 'get_time', 'arguments': {'timezone': 'Africa/Cairo'}}]
2024-05-03 02:35:24 INFO semantic_router.utils.logger Document 1 length: 38
2024-05-03 02:35:24 INFO semantic_router.utils.logger Document 1 trunc length: 38


...

>>> FAIL <<<.
Failed to process: I need to know the time in Beijing. with error: 'No time zone found with key Asia/Beijing'
Processing utterance:  Show me the current time in Seoul.


2024-05-03 02:36:29 INFO semantic_router.utils.logger Extracting function input...
2024-05-03 02:36:30 INFO semantic_router.utils.logger LLM output: [
    {
        "function_name": "get_time",
        "arguments": {
            "timezone": "Asia/Seoul"
        }
    }
]
2024-05-03 02:36:30 INFO semantic_router.utils.logger Function inputs: [{'function_name': 'get_time', 'arguments': {'timezone': 'Asia/Seoul'}}]
2024-05-03 02:36:30 INFO semantic_router.utils.logger Document 1 length: 29
2024-05-03 02:36:30 INFO semantic_router.utils.logger Document 1 trunc length: 29


07:36
>>> PASS <<<.
Processing utterance:  What time is it in Singapore?


2024-05-03 02:36:30 INFO semantic_router.utils.logger Extracting function input...
2024-05-03 02:36:31 INFO semantic_router.utils.logger LLM output: [
    {
        "function_name": "get_time",
        "arguments": {
            "timezone": "Asia/Singapore"
        }
    }
]
2024-05-03 02:36:31 INFO semantic_router.utils.logger Function inputs: [{'function_name': 'get_time', 'arguments': {'timezone': 'Asia/Singapore'}}]


06:36
>>> PASS <<<.
Number of successful responses: 48
Number of failed responses: 2


### Test `get_time_difference` Function Argument Extraction

In [20]:
test_function_calling(
    utterances=test_utterances_get_time_difference,
    function_name='get_time_difference',
    expected_num_responses=1
    )

2024-05-03 02:36:31 INFO semantic_router.utils.logger Document 1 length: 55
2024-05-03 02:36:31 INFO semantic_router.utils.logger Document 1 trunc length: 55


Processing utterance:  What's the hour difference between Vancouver and Miami?


2024-05-03 02:36:32 INFO semantic_router.utils.logger Extracting function input...
2024-05-03 02:36:33 INFO semantic_router.utils.logger LLM output: [
    {
        "function_name": "get_time_difference",
        "arguments": {
            "timezone1": "America/Vancouver",
            "timezone2": "America/New_York"
        }
    }
]
2024-05-03 02:36:33 INFO semantic_router.utils.logger Function inputs: [{'function_name': 'get_time_difference', 'arguments': {'timezone1': 'America/Vancouver', 'timezone2': 'America/New_York'}}]
2024-05-03 02:36:33 INFO semantic_router.utils.logger Document 1 length: 60
2024-05-03 02:36:33 INFO semantic_router.utils.logger Document 1 trunc length: 60


The time difference between America/Vancouver and America/New_York is 3.0 hours.
>>> PASS <<<.
Processing utterance:  Can you calculate the hours separating Helsinki from Denver?


2024-05-03 02:36:33 INFO semantic_router.utils.logger Extracting function input...
2024-05-03 02:36:35 INFO semantic_router.utils.logger LLM output: [
    {
        "function_name": "get_time_difference",
        "arguments": {
            "timezone1": "Europe/Helsinki",
            "timezone2": "America/Denver"
        }
    }
]
2024-05-03 02:36:35 INFO semantic_router.utils.logger Function inputs: [{'function_name': 'get_time_difference', 'arguments': {'timezone1': 'Europe/Helsinki', 'timezone2': 'America/Denver'}}]
2024-05-03 02:36:35 INFO semantic_router.utils.logger Document 1 length: 56
2024-05-03 02:36:35 INFO semantic_router.utils.logger Document 1 trunc length: 56


...

The time difference between Europe/Copenhagen and America/Toronto is -6.0 hours.
>>> PASS <<<.
Processing utterance:  Can you find the time gap from Oslo to Rome?


2024-05-03 02:37:52 INFO semantic_router.utils.logger Extracting function input...
2024-05-03 02:37:53 INFO semantic_router.utils.logger LLM output: [
    {
        "function_name": "get_time_difference",
        "arguments": {
            "timezone1": "Europe/Oslo",
            "timezone2": "Europe/Rome"
        }
    }
]
2024-05-03 02:37:53 INFO semantic_router.utils.logger Function inputs: [{'function_name': 'get_time_difference', 'arguments': {'timezone1': 'Europe/Oslo', 'timezone2': 'Europe/Rome'}}]
2024-05-03 02:37:53 INFO semantic_router.utils.logger Document 1 length: 52
2024-05-03 02:37:53 INFO semantic_router.utils.logger Document 1 trunc length: 52


The time difference between Europe/Oslo and Europe/Rome is 0.0 hours.
>>> PASS <<<.
Processing utterance:  Tell me the hours separating Montreal and Las Vegas.


2024-05-03 02:37:53 INFO semantic_router.utils.logger Extracting function input...
2024-05-03 02:37:55 INFO semantic_router.utils.logger LLM output: [
    {
        "function_name": "get_time_difference",
        "arguments": {
            "timezone1": "America/Montreal",
            "timezone2": "America/Los_Angeles"
        }
    }
]
2024-05-03 02:37:55 INFO semantic_router.utils.logger Function inputs: [{'function_name': 'get_time_difference', 'arguments': {'timezone1': 'America/Montreal', 'timezone2': 'America/Los_Angeles'}}]


The time difference between America/Montreal and America/Los_Angeles is -3.0 hours.
>>> PASS <<<.
Number of successful responses: 47
Number of failed responses: 3


### Test `convert_time` Function Argument Extraction

In [21]:
test_function_calling(
    utterances=test_utterances_convert_time,
    function_name='convert_time',
    expected_num_responses=1
    )

2024-05-03 02:37:55 INFO semantic_router.utils.logger Document 1 length: 56
2024-05-03 02:37:55 INFO semantic_router.utils.logger Document 1 trunc length: 56


Processing utterance:  Please switch 14:30 from Los Angeles time to Tokyo time.


2024-05-03 02:37:55 INFO semantic_router.utils.logger Extracting function input...
2024-05-03 02:37:56 INFO semantic_router.utils.logger LLM output: [
    {
        "function_name": "convert_time",
        "arguments": {
            "time": "14:30",
            "from_timezone": "America/Los_Angeles",
            "to_timezone": "Asia/Tokyo"
        }
    }
]
2024-05-03 02:37:56 INFO semantic_router.utils.logger Function inputs: [{'function_name': 'convert_time', 'arguments': {'time': '14:30', 'from_timezone': 'America/Los_Angeles', 'to_timezone': 'Asia/Tokyo'}}]
2024-05-03 02:37:56 INFO semantic_router.utils.logger Document 1 length: 47
2024-05-03 02:37:56 INFO semantic_router.utils.logger Document 1 trunc length: 47


06:30
>>> PASS <<<.
Processing utterance:  Modify 10:15 from London time to New York time.


2024-05-03 02:37:57 INFO semantic_router.utils.logger Extracting function input...
2024-05-03 02:37:58 INFO semantic_router.utils.logger LLM output: [
    {
        "function_name": "convert_time",
        "arguments": {
            "time": "10:15",
            "from_timezone": "Europe/London",
            "to_timezone": "America/New_York"
        }
    }
]
2024-05-03 02:37:58 INFO semantic_router.utils.logger Function inputs: [{'function_name': 'convert_time', 'arguments': {'time': '10:15', 'from_timezone': 'Europe/London', 'to_timezone': 'America/New_York'}}]
2024-05-03 02:37:58 INFO semantic_router.utils.logger Document 1 length: 46
2024-05-03 02:37:58 INFO semantic_router.utils.logger Document 1 trunc length: 46


...

05:40
>>> PASS <<<.
Processing utterance:  Reestablish 12:00 from Toronto time to Lisbon time.


2024-05-03 02:39:19 INFO semantic_router.utils.logger Extracting function input...
2024-05-03 02:39:20 INFO semantic_router.utils.logger LLM output: [
    {
        "function_name": "convert_time",
        "arguments": {
            "time": "12:00",
            "from_timezone": "America/Toronto",
            "to_timezone": "Europe/Lisbon"
        }
    }
]
2024-05-03 02:39:20 INFO semantic_router.utils.logger Function inputs: [{'function_name': 'convert_time', 'arguments': {'time': '12:00', 'from_timezone': 'America/Toronto', 'to_timezone': 'Europe/Lisbon'}}]
2024-05-03 02:39:20 INFO semantic_router.utils.logger Document 1 length: 56
2024-05-03 02:39:20 INFO semantic_router.utils.logger Document 1 trunc length: 56


17:00
>>> PASS <<<.
Processing utterance:  Reformulate 20:00 from Moscow time to Buenos Aires time.


2024-05-03 02:39:20 INFO semantic_router.utils.logger Extracting function input...
2024-05-03 02:39:22 INFO semantic_router.utils.logger LLM output: [
    {
        "function_name": "convert_time",
        "arguments": {
            "time": "20:00",
            "from_timezone": "Europe/Moscow",
            "to_timezone": "America/Argentina/Buenos_Aires"
        }
    }
]
2024-05-03 02:39:22 INFO semantic_router.utils.logger Function inputs: [{'function_name': 'convert_time', 'arguments': {'time': '20:00', 'from_timezone': 'Europe/Moscow', 'to_timezone': 'America/Argentina/Buenos_Aires'}}]


14:00
>>> PASS <<<.
Number of successful responses: 50
Number of failed responses: 0


### Test `get_time` and `get_time_difference` Function Argument Extractions

In [22]:
# import re

# success_count = 0
# fail_count = 0

# time_pattern = r"\d{2}:\d{2}"
# time_diff_pattern = r"The time difference between .+ and .+ is -?\d+(\.\d+)? hours\."

# for utterance in test_utterances_get_time_and_get_time_difference:
#     print('='*100)
#     print(u"Processing utterance: ", utterance)
#     try:
#         results = rl(utterance)
#         parsed_results = parse_response(results)
#         # Check if the parsed_results match the required formats
#         if re.match(time_pattern, parsed_results[0]) and re.match(time_diff_pattern, parsed_results[1]):
#             success_count += 1
#             print('Pass.')
#         else:
#             print(f"Invalid response format: {results}")
#             fail_count += 1
#     except Exception as e:
#         print(f"Failed to process: {utterance} with error: {e}")
#         fail_count += 1
#     print('='*100)
# print('='*100)
# print(f"Number of successful responses: {success_count}")
# print(f"Number of failed responses: {fail_count}")
# print('='*100)


In [23]:
test_function_calling(
    utterances=test_utterances_get_time_and_get_time_difference,
    function_name='get_time_and_get_time_difference',
    expected_num_responses=2
    )

2024-05-03 02:39:22 INFO semantic_router.utils.logger Document 1 length: 73
2024-05-03 02:39:22 INFO semantic_router.utils.logger Document 1 trunc length: 73


Processing utterance:  What's the current time in Vancouver and how far behind is it from Tokyo?


2024-05-03 02:39:22 INFO semantic_router.utils.logger Extracting function input...
2024-05-03 02:39:24 INFO semantic_router.utils.logger LLM output: [
    {
        "function_name": "get_time",
        "arguments": {
            "timezone": "America/Vancouver"
        }
    },
    {
        "function_name": "get_time_difference",
        "arguments": {
            "timezone1": "America/Vancouver",
            "timezone2": "Asia/Tokyo"
        }
    }
]
2024-05-03 02:39:24 INFO semantic_router.utils.logger Function inputs: [{'function_name': 'get_time', 'arguments': {'timezone': 'America/Vancouver'}}, {'function_name': 'get_time_difference', 'arguments': {'timezone1': 'America/Vancouver', 'timezone2': 'Asia/Tokyo'}}]
2024-05-03 02:39:24 INFO semantic_router.utils.logger Document 1 length: 79
2024-05-03 02:39:24 INFO semantic_router.utils.logger Document 1 trunc length: 79


15:39
The time difference between America/Vancouver and Asia/Tokyo is 16.0 hours.
>>> PASS <<<.
Processing utterance:  Please tell me the current time in Helsinki and the hour difference with Cairo.


2024-05-03 02:39:24 INFO semantic_router.utils.logger Extracting function input...
2024-05-03 02:39:26 INFO semantic_router.utils.logger LLM output: [
    {
        "function_name": "get_time",
        "arguments": {
            "timezone": "Europe/Helsinki"
        }
    },
    {
        "function_name": "get_time_difference",
        "arguments": {
            "timezone1": "Europe/Helsinki",
            "timezone2": "Africa/Cairo"
        }
    }
]
2024-05-03 02:39:26 INFO semantic_router.utils.logger Function inputs: [{'function_name': 'get_time', 'arguments': {'timezone': 'Europe/Helsinki'}}, {'function_name': 'get_time_difference', 'arguments': {'timezone1': 'Europe/Helsinki', 'timezone2': 'Africa/Cairo'}}]
2024-05-03 02:39:26 INFO semantic_router.utils.logger Document 1 length: 80
2024-05-03 02:39:26 INFO semantic_router.utils.logger Document 1 trunc length: 80


...

23:41
The time difference between Africa/Casablanca and Asia/Shanghai is 7.0 hours.
>>> PASS <<<.
Processing utterance:  Can you provide the current time in Accra and how many hours it is behind Paris?


2024-05-03 02:41:06 INFO semantic_router.utils.logger Extracting function input...
2024-05-03 02:41:08 INFO semantic_router.utils.logger LLM output: [
    {
        "function_name": "get_time",
        "arguments": {
            "timezone": "Africa/Accra"
        }
    },
    {
        "function_name": "get_time_difference",
        "arguments": {
            "timezone1": "Africa/Accra",
            "timezone2": "Europe/Paris"
        }
    }
]
2024-05-03 02:41:08 INFO semantic_router.utils.logger Function inputs: [{'function_name': 'get_time', 'arguments': {'timezone': 'Africa/Accra'}}, {'function_name': 'get_time_difference', 'arguments': {'timezone1': 'Africa/Accra', 'timezone2': 'Europe/Paris'}}]
2024-05-03 02:41:08 INFO semantic_router.utils.logger Document 1 length: 82
2024-05-03 02:41:08 INFO semantic_router.utils.logger Document 1 trunc length: 82


22:41
The time difference between Africa/Accra and Europe/Paris is 2.0 hours.
>>> PASS <<<.
Processing utterance:  What is the current time in Algiers and how many hours ahead is it from Vancouver?


2024-05-03 02:41:08 INFO semantic_router.utils.logger Extracting function input...
2024-05-03 02:41:10 INFO semantic_router.utils.logger LLM output: [
    {
        "function_name": "get_time",
        "arguments": {
            "timezone": "Africa/Algiers"
        }
    },
    {
        "function_name": "get_time_difference",
        "arguments": {
            "timezone1": "Africa/Algiers",
            "timezone2": "America/Vancouver"
        }
    }
]
2024-05-03 02:41:10 INFO semantic_router.utils.logger Function inputs: [{'function_name': 'get_time', 'arguments': {'timezone': 'Africa/Algiers'}}, {'function_name': 'get_time_difference', 'arguments': {'timezone1': 'Africa/Algiers', 'timezone2': 'America/Vancouver'}}]


23:41
The time difference between Africa/Algiers and America/Vancouver is -8.0 hours.
>>> PASS <<<.
Number of successful responses: 49
Number of failed responses: 1


### Test `get_time` and `convert_time` Function Argument Extractions

In [24]:
# import re

# success_count = 0
# fail_count = 0

# time_pattern = r"\d{2}:\d{2}"
# time_diff_pattern = r"The time difference between .+ and .+ is -?\d+(\.\d+)? hours\."

# for utterance in test_utterances_get_time_and_convert_time:
#     print('='*100)
#     print(u"Processing utterance: ", utterance)
#     try:
#         results = rl(utterance)
#         parsed_results = parse_response(results)
#         # Check if the parsed_results match the required formats
#         if re.match(time_pattern, parsed_results[0]) and re.match(time_diff_pattern, parsed_results[1]):
#             success_count += 1
#             print('Pass.')
#         else:
#             print(f"Invalid response format: {results}")
#             fail_count += 1
#     except Exception as e:
#         print(f"Failed to process: {utterance} with error: {e}")
#         fail_count += 1
#     print('='*100)
# print('='*100)
# print(f"Number of successful responses: {success_count}")
# print(f"Number of failed responses: {fail_count}")
# print('='*100)


In [25]:
test_function_calling(
    utterances=test_utterances_get_time_and_convert_time,
    function_name='get_time_and_convert_time',
    expected_num_responses=2
    )

2024-05-03 02:41:10 INFO semantic_router.utils.logger Document 1 length: 106
2024-05-03 02:41:10 INFO semantic_router.utils.logger Document 1 trunc length: 106


Processing utterance:  Could you provide the current time in Vancouver? Additionally, what is 15:00 Vancouver time in Tokyo time?


2024-05-03 02:41:10 INFO semantic_router.utils.logger Extracting function input...
2024-05-03 02:41:12 INFO semantic_router.utils.logger LLM output: [
    {
        "function_name": "get_time",
        "arguments": {
            "timezone": "America/Vancouver"
        }
    },
    {
        "function_name": "convert_time",
        "arguments": {
            "time": "15:00",
            "from_timezone": "America/Vancouver",
            "to_timezone": "Asia/Tokyo"
        }
    }
]
2024-05-03 02:41:12 INFO semantic_router.utils.logger Function inputs: [{'function_name': 'get_time', 'arguments': {'timezone': 'America/Vancouver'}}, {'function_name': 'convert_time', 'arguments': {'time': '15:00', 'from_timezone': 'America/Vancouver', 'to_timezone': 'Asia/Tokyo'}}]
2024-05-03 02:41:12 INFO semantic_router.utils.logger Document 1 length: 91
2024-05-03 02:41:12 INFO semantic_router.utils.logger Document 1 trunc length: 91


15:41
07:00
>>> PASS <<<.
Processing utterance:  What's the current time in Oslo? And can you convert 14:00 from Oslo time to New York time?


2024-05-03 02:41:12 INFO semantic_router.utils.logger Extracting function input...
2024-05-03 02:41:14 INFO semantic_router.utils.logger LLM output: [
    {
        "function_name": "get_time",
        "arguments": {
            "timezone": "Europe/Oslo"
        }
    },
    {
        "function_name": "convert_time",
        "arguments": {
            "time": "14:00",
            "from_timezone": "Europe/Oslo",
            "to_timezone": "America/New_York"
        }
    }
]
2024-05-03 02:41:14 INFO semantic_router.utils.logger Function inputs: [{'function_name': 'get_time', 'arguments': {'timezone': 'Europe/Oslo'}}, {'function_name': 'convert_time', 'arguments': {'time': '14:00', 'from_timezone': 'Europe/Oslo', 'to_timezone': 'America/New_York'}}]
2024-05-03 02:41:14 INFO semantic_router.utils.logger Document 1 length: 93
2024-05-03 02:41:14 INFO semantic_router.utils.logger Document 1 trunc length: 93


...

00:42
19:50
>>> PASS <<<.
Processing utterance:  Show the current time in Glasgow. Convert 11:15 from Glasgow time to New Delhi time.


2024-05-03 02:42:59 INFO semantic_router.utils.logger Extracting function input...
2024-05-03 02:43:01 INFO semantic_router.utils.logger LLM output: [
    {
        "function_name": "get_time",
        "arguments": {
            "timezone": "Europe/London"
        }
    },
    {
        "function_name": "convert_time",
        "arguments": {
            "time": "11:15",
            "from_timezone": "Europe/London",
            "to_timezone": "Asia/Kolkata"
        }
    }
]
2024-05-03 02:43:01 INFO semantic_router.utils.logger Function inputs: [{'function_name': 'get_time', 'arguments': {'timezone': 'Europe/London'}}, {'function_name': 'convert_time', 'arguments': {'time': '11:15', 'from_timezone': 'Europe/London', 'to_timezone': 'Asia/Kolkata'}}]
2024-05-03 02:43:01 INFO semantic_router.utils.logger Document 1 length: 87
2024-05-03 02:43:01 INFO semantic_router.utils.logger Document 1 trunc length: 87


23:43
15:45
>>> PASS <<<.
Processing utterance:  Can you find the current time in Caracas? Switch 16:00 from Caracas time to Tokyo time.


2024-05-03 02:43:01 INFO semantic_router.utils.logger Extracting function input...
2024-05-03 02:43:03 INFO semantic_router.utils.logger LLM output: [
    {
        "function_name": "get_time",
        "arguments": {
            "timezone": "America/Caracas"
        }
    },
    {
        "function_name": "convert_time",
        "arguments": {
            "time": "16:00",
            "from_timezone": "America/Caracas",
            "to_timezone": "Asia/Tokyo"
        }
    }
]
2024-05-03 02:43:03 INFO semantic_router.utils.logger Function inputs: [{'function_name': 'get_time', 'arguments': {'timezone': 'America/Caracas'}}, {'function_name': 'convert_time', 'arguments': {'time': '16:00', 'from_timezone': 'America/Caracas', 'to_timezone': 'Asia/Tokyo'}}]


18:43
05:00
>>> PASS <<<.
Number of successful responses: 46
Number of failed responses: 4


### Test `get_time_difference` and `convert_time` Function Argument Extractions

In [26]:
test_function_calling(
    utterances=test_utterances_get_time_difference_and_convert_time,
    function_name='get_time_difference_and_convert_time',
    expected_num_responses=2
    )

2024-05-03 02:43:03 INFO semantic_router.utils.logger Document 1 length: 115
2024-05-03 02:43:03 INFO semantic_router.utils.logger Document 1 trunc length: 115


Processing utterance:  What's the time difference between Los Angeles and Tokyo, and can you convert 10:00 from Los Angeles to Tokyo time?


2024-05-03 02:43:04 INFO semantic_router.utils.logger Extracting function input...
2024-05-03 02:43:06 INFO semantic_router.utils.logger LLM output: [
    {
        "function_name": "get_time_difference",
        "arguments": {
            "timezone1": "America/Los_Angeles",
            "timezone2": "Asia/Tokyo"
        }
    },
    {
        "function_name": "convert_time",
        "arguments": {
            "time": "10:00",
            "from_timezone": "America/Los_Angeles",
            "to_timezone": "Asia/Tokyo"
        }
    }
]
2024-05-03 02:43:06 INFO semantic_router.utils.logger Function inputs: [{'function_name': 'get_time_difference', 'arguments': {'timezone1': 'America/Los_Angeles', 'timezone2': 'Asia/Tokyo'}}, {'function_name': 'convert_time', 'arguments': {'time': '10:00', 'from_timezone': 'America/Los_Angeles', 'to_timezone': 'Asia/Tokyo'}}]
2024-05-03 02:43:06 INFO semantic_router.utils.logger Document 1 length: 107
2024-05-03 02:43:06 INFO semantic_router.utils.logger D

The time difference between America/Los_Angeles and Asia/Tokyo is 16.0 hours.
02:00
>>> PASS <<<.
Processing utterance:  Please find out how many hours ahead London is from New York and switch 14:30 from London to New York time.


2024-05-03 02:43:07 INFO semantic_router.utils.logger Extracting function input...
2024-05-03 02:43:09 INFO semantic_router.utils.logger LLM output: [
    {
        "function_name": "get_time_difference",
        "arguments": {
            "timezone1": "America/New_York",
            "timezone2": "Europe/London"
        }
    },
    {
        "function_name": "convert_time",
        "arguments": {
            "time": "14:30",
            "from_timezone": "Europe/London",
            "to_timezone": "America/New_York"
        }
    }
]
2024-05-03 02:43:09 INFO semantic_router.utils.logger Function inputs: [{'function_name': 'get_time_difference', 'arguments': {'timezone1': 'America/New_York', 'timezone2': 'Europe/London'}}, {'function_name': 'convert_time', 'arguments': {'time': '14:30', 'from_timezone': 'Europe/London', 'to_timezone': 'America/New_York'}}]
2024-05-03 02:43:09 INFO semantic_router.utils.logger Document 1 length: 100
2024-05-03 02:43:09 INFO semantic_router.utils.logger D

...

The time difference between Europe/Helsinki and America/Chicago is -8.0 hours.
23:30
>>> PASS <<<.
Processing utterance:  What is the time difference between Prague and Toronto, and can you switch 09:50 Prague time to Toronto time?


2024-05-03 02:45:06 INFO semantic_router.utils.logger Extracting function input...
2024-05-03 02:45:08 INFO semantic_router.utils.logger LLM output: [
    {
        "function_name": "get_time_difference",
        "arguments": {
            "timezone1": "Europe/Prague",
            "timezone2": "America/Toronto"
        }
    },
    {
        "function_name": "convert_time",
        "arguments": {
            "time": "09:50",
            "from_timezone": "Europe/Prague",
            "to_timezone": "America/Toronto"
        }
    }
]
2024-05-03 02:45:08 INFO semantic_router.utils.logger Function inputs: [{'function_name': 'get_time_difference', 'arguments': {'timezone1': 'Europe/Prague', 'timezone2': 'America/Toronto'}}, {'function_name': 'convert_time', 'arguments': {'time': '09:50', 'from_timezone': 'Europe/Prague', 'to_timezone': 'America/Toronto'}}]
2024-05-03 02:45:08 INFO semantic_router.utils.logger Document 1 length: 103
2024-05-03 02:45:08 INFO semantic_router.utils.logger Docum

The time difference between Europe/Prague and America/Toronto is -6.0 hours.
03:50
>>> PASS <<<.
Processing utterance:  Measure the time gap between Budapest and Melbourne, and convert 11:55 Budapest time to Melbourne time.


2024-05-03 02:45:08 INFO semantic_router.utils.logger Extracting function input...
2024-05-03 02:45:10 INFO semantic_router.utils.logger LLM output: [
    {
        "function_name": "get_time_difference",
        "arguments": {
            "timezone1": "Europe/Budapest",
            "timezone2": "Australia/Melbourne"
        }
    },
    {
        "function_name": "convert_time",
        "arguments": {
            "time": "11:55",
            "from_timezone": "Europe/Budapest",
            "to_timezone": "Australia/Melbourne"
        }
    }
]
2024-05-03 02:45:10 INFO semantic_router.utils.logger Function inputs: [{'function_name': 'get_time_difference', 'arguments': {'timezone1': 'Europe/Budapest', 'timezone2': 'Australia/Melbourne'}}, {'function_name': 'convert_time', 'arguments': {'time': '11:55', 'from_timezone': 'Europe/Budapest', 'to_timezone': 'Australia/Melbourne'}}]


The time difference between Europe/Budapest and Australia/Melbourne is 8.0 hours.
19:55
>>> PASS <<<.
Number of successful responses: 50
Number of failed responses: 0


### Test All Function Argument Extractions

In [27]:
test_function_calling(
    utterances=test_utterances_all_functions,
    function_name='all_functions',
    expected_num_responses=3
    )

2024-05-03 02:45:10 INFO semantic_router.utils.logger Document 1 length: 136
2024-05-03 02:45:10 INFO semantic_router.utils.logger Document 1 trunc length: 136


Processing utterance:  What's the current time in Atlanta? How many hours is Atlanta ahead of Vancouver? And convert 14:20 from Atlanta time to Vancouver time.


2024-05-03 02:45:11 INFO semantic_router.utils.logger Extracting function input...
2024-05-03 02:45:14 INFO semantic_router.utils.logger LLM output: [
    {
        "function_name": "get_time",
        "arguments": {
            "timezone": "America/New_York"
        }
    },
    {
        "function_name": "get_time_difference",
        "arguments": {
            "timezone1": "America/New_York",
            "timezone2": "Vancouver"
        }
    },
    {
        "function_name": "convert_time",
        "arguments": {
            "time": "14:20",
            "from_timezone": "America/New_York",
            "to_timezone": "Vancouver"
        }
    }
]
2024-05-03 02:45:14 INFO semantic_router.utils.logger Function inputs: [{'function_name': 'get_time', 'arguments': {'timezone': 'America/New_York'}}, {'function_name': 'get_time_difference', 'arguments': {'timezone1': 'America/New_York', 'timezone2': 'Vancouver'}}, {'function_name': 'convert_time', 'arguments': {'time': '14:20', 'from_timez

18:45
>>> FAIL <<<.
Failed to process: What's the current time in Atlanta? How many hours is Atlanta ahead of Vancouver? And convert 14:20 from Atlanta time to Vancouver time. with error: 'No time zone found with key Vancouver'
Processing utterance:  Please tell me the current time in Boston, the time difference with Zurich, and change 16:00 Boston time to Zurich time.


2024-05-03 02:45:14 INFO semantic_router.utils.logger Extracting function input...
2024-05-03 02:45:17 INFO semantic_router.utils.logger LLM output: [
    {
        "function_name": "get_time",
        "arguments": {
            "timezone": "America/New_York"
        }
    },
    {
        "function_name": "get_time_difference",
        "arguments": {
            "timezone1": "Boston",
            "timezone2": "Zurich"
        }
    },
    {
        "function_name": "convert_time",
        "arguments": {
            "time": "16:00",
            "from_timezone": "America/New_York",
            "to_timezone": "Zurich"
        }
    }
]
2024-05-03 02:45:17 INFO semantic_router.utils.logger Function inputs: [{'function_name': 'get_time', 'arguments': {'timezone': 'America/New_York'}}, {'function_name': 'get_time_difference', 'arguments': {'timezone1': 'Boston', 'timezone2': 'Zurich'}}, {'function_name': 'convert_time', 'arguments': {'time': '16:00', 'from_timezone': 'America/New_York', 'to

...

23:47
The time difference between Europe/London and Asia/Kolkata is 4.5 hours.
17:15
>>> PASS <<<.
Processing utterance:  Tell me the current time in Manchester, how many hours behind is Manchester from Dubai, and adjust 14:30 Manchester time to Dubai time.


2024-05-03 02:47:43 INFO semantic_router.utils.logger Extracting function input...
2024-05-03 02:47:45 INFO semantic_router.utils.logger LLM output: [
    {
        "function_name": "get_time",
        "arguments": {
            "timezone": "Europe/London"
        }
    },
    {
        "function_name": "get_time_difference",
        "arguments": {
            "timezone1": "Europe/London",
            "timezone2": "Asia/Dubai"
        }
    },
    {
        "function_name": "convert_time",
        "arguments": {
            "time": "14:30",
            "from_timezone": "Europe/London",
            "to_timezone": "Asia/Dubai"
        }
    }
]
2024-05-03 02:47:45 INFO semantic_router.utils.logger Function inputs: [{'function_name': 'get_time', 'arguments': {'timezone': 'Europe/London'}}, {'function_name': 'get_time_difference', 'arguments': {'timezone1': 'Europe/London', 'timezone2': 'Asia/Dubai'}}, {'function_name': 'convert_time', 'arguments': {'time': '14:30', 'from_timezone': 'Europ

23:47
The time difference between Europe/London and Asia/Dubai is 3.0 hours.
17:30
>>> PASS <<<.
Processing utterance:  What is the current time in Liverpool? What is the time difference between Liverpool and Beijing? And convert 16:15 Liverpool time to Beijing time.


2024-05-03 02:47:46 INFO semantic_router.utils.logger Extracting function input...
2024-05-03 02:47:48 INFO semantic_router.utils.logger LLM output: [
    {
        "function_name": "get_time",
        "arguments": {
            "timezone": "Europe/London"
        }
    },
    {
        "function_name": "get_time_difference",
        "arguments": {
            "timezone1": "Europe/London",
            "timezone2": "Asia/Shanghai"
        }
    },
    {
        "function_name": "convert_time",
        "arguments": {
            "time": "16:15",
            "from_timezone": "Europe/London",
            "to_timezone": "Asia/Shanghai"
        }
    }
]
2024-05-03 02:47:48 INFO semantic_router.utils.logger Function inputs: [{'function_name': 'get_time', 'arguments': {'timezone': 'Europe/London'}}, {'function_name': 'get_time_difference', 'arguments': {'timezone1': 'Europe/London', 'timezone2': 'Asia/Shanghai'}}, {'function_name': 'convert_time', 'arguments': {'time': '16:15', 'from_timezone

23:47
The time difference between Europe/London and Asia/Shanghai is 7.0 hours.
23:15
>>> PASS <<<.
Number of successful responses: 44
Number of failed responses: 6


In [ ]:
        prompt = f"""
You are an accurate and reliable computer program that only outputs valid JSON. 
Your task is to:
    1) Pick the most relevant Python function schema(s) from FUNCTION_SCHEMAS below, based on the input QUERY.
    2) Output JSON representing the input arguments of the chosen function schema(s), where the input argument values are ascertained from information in the QUERY.

These are the Python functions' schema:

### FUNCTION_SCHEMAS Start ###
    {json.dumps(function_schemas, indent=4)}
### FUNCTION_SCHEMAS End ###

This is the input query.

### QUERY Start ###
    {query}
### QUERY End ###

Steps:
    1) Pick relevant Python FUNCTION_SCHEMAS based on the following criteria:
        - The function output provides information that is requested in the QUERY.
        - The information that is required as input to the function is present in the QUERY, or can easily be ascertained from the QUERY.
    2) Output the input arguments of the chosen function in valid JSON format (see FORMATTING_INSTRUCTIONS below).
Note: Pay special attention to FUNCTION_SCHEMA "description" and "signature" fields to determine the correct format of the input arguments.

### FORMATTING_INSTRUCTIONS Start ###
    Return a response in valid JSON format. Do not return any other explanation, text or tokens, just the JSON.
    The JSON should be in the following format:
    [
        {{
            "function_name": "FUNCTION_NAME",
            "arguments": {{
                "ARGUMENT_NAME": "ARGUMENT_VALUE_FROM_QUERY",
                "ARGUMENT_NAME": "ARGUMENT_VALUE_FROM_QUERY",
                ...
            }}
        }},
        {{
            "function_name": "FUNCTION_NAME",
            "arguments": {{
                "ARGUMENT_NAME": "ARGUMENT_VALUE_FROM_QUERY",
                "ARGUMENT_NAME": "ARGUMENT_VALUE_FROM_QUERY",
                ...
            }}
        }},
        ...
    ]
    NOTE: The same function might be represented in the JSON multiple times, with different arguments, if the function is relevant for different parts of the query.
### FORMATTING_INSTRUCTIONS End ###

I will tip $500 for an accurate JSON output. You will be penalized for an inaccurate JSON output.

Provide JSON output now:
    """